### Import Data Sets from the following sources and transform to one record per zip code:

* 2016 Census population estimates by zip code from factfinder.census.gov
* OSHPD hospital and emergency room utilization data by zip code from oshpd.ca.gov
* CA Area Deprivation Index by zip code from University of Wisconsin School of Medicine and Public Health
* ESRI Tapestry consumer segmentation data by zip code from ESRI / ArcGIS
* Schedule drug metrics by zip code from  ...

### Area Deprivation Index (https://www.neighborhoodatlas.medicine.wisc.edu/)
The Area Deprivation Index is used as an indicator of socioeconomic deprivation, which we will demonstrate is a key driver of high emergent and inpatient healthcare utilization.  Originally created by the Health Resources and Services Administration nearly three decades ago, the ADI is composed of 17 education, employment, housing-quality, and poverty measures originally drawn from long-form Census data and updated by Dr. Amy Kind’s research team at the University of Wisconsin School of Medicine and Public Health to incorporate more recent American Community Survey (ACS) data.

### Merge data sets into one dataframe and calculate ratios:

* Hosp and ED util per 1,000 population
* Schedule 2-4 controlled substance utilization per 1,000 population
* Total Rx utilization per 1,000 population

## Import Python Modules and all Data Sets

In [215]:
import pandas as pd
import numpy as np

In [216]:
# Read 2016 census population estimates with some age/sex detail
ca_census_demo_df = pd.read_csv("Resources/ACS_16_5YR_DP05_with_ann.csv",skiprows=1)
ca_census_demo_df = ca_census_demo_df.rename(columns={'Id2':'Zip','Estimate; SEX AND AGE - Total population':'Total Pop',
                                                      'Estimate; SEX AND AGE - Total population - Male':'Male Pop',
                                                      'Percent; SEX AND AGE - Total population - Male':'Pct Male',
                                                      'Estimate; SEX AND AGE - Total population - Female':'Female Pop',
                                                      'Percent; SEX AND AGE - Total population - Female':'Pct Female'})
ca_census_demo_df = ca_census_demo_df[['Zip','Total Pop','Male Pop','Pct Male','Female Pop','Pct Female']]

# drop non-CA zips
ca_census_demo_df = ca_census_demo_df.drop([0,1,2,3,4,5]) # NV zips
ca_census_demo_df.head()

,Zip,Total Pop,Male Pop,Pct Male,Female Pop,Pct Female
6,90001,57942,29520,50.9,28422,49.1
7,90002,51826,24877,48.0,26949,52.0
8,90003,70208,33953,48.4,36255,51.6
9,90004,63095,31141,49.4,31954,50.6
10,90005,39338,19592,49.8,19746,50.2


In [217]:
# Read 2016 CA OSHPD Utilization file
ca_hosp_2016_df = pd.read_csv("Resources/POMS2016.csv")

# Limit to data that can be aggregated by pt zip code
ca_hosp_2016_df = ca_hosp_2016_df[['pzip','year','pattype','discharges']]
ca_hosp_2016_df = ca_hosp_2016_df.rename(columns={'year':'Year'})

# Use groupby to summarize discharges by pzip, pcounty, year and pattype
ca_hosp_2016_df = ca_hosp_2016_df.groupby(['pzip','Year','pattype']).sum()
ca_hosp_2016_df = ca_hosp_2016_df.reset_index()
ca_hosp_2016_df = ca_hosp_2016_df.set_index(['pzip','Year'])
ca_hosp_2016_df = ca_hosp_2016_df.pivot_table(index=['pzip','Year'], columns='pattype',values='discharges').reset_index()

# drop records with no zip code (ie. ARIZONA, HOMELESS, etc in pzip column)
ca_hosp_2016_df = ca_hosp_2016_df.drop([2672,2673,2674,2675,2676,2677,2678])

# convert pzip from string to int
ca_hosp_2016_df['pzip'] = ca_hosp_2016_df['pzip'].apply(pd.to_numeric)

#ca_hosp_2016_df.count()
ca_hosp_2016_df.head()

pattype,pzip,Year,AS Only,ED Only,Inpatient,Inpatient from ED
0,90001,2016,1798.0,24554.0,3628.0,3286.0
1,90002,2016,1661.0,25168.0,3421.0,3314.0
2,90003,2016,2111.0,32061.0,4610.0,4648.0
3,90004,2016,2115.0,15820.0,2751.0,2639.0
4,90005,2016,1107.0,8650.0,1579.0,1488.0


In [218]:
drug_sched_df = pd.read_csv("Resources/schedule_drugs_table9_presciptions_by_year_locale_drugschedule.csv")

# create dataframe with only 2016 drug data
drug_sched_2016_df = drug_sched_df.loc[drug_sched_df['xYear']==2016]

drug_sched_2016_df = drug_sched_2016_df[['Zip','County','State','xYear','Schedule_Group','RxCount']]
drug_sched_2016_df = drug_sched_2016_df.set_index(['Zip','County','State','xYear'])
drug_sched_2016_df = drug_sched_2016_df.pivot_table(index=['Zip','County','State','xYear'], columns='Schedule_Group', values='RxCount').reset_index()
drug_sched_2016_df = drug_sched_2016_df.rename(columns={'2':'C2','3':'C3','4':'C4','All':'C2-C4'})
#drug_sched_2016_df['C2 per 1,000'] = drug_sched_2016_df['C2']/drug_sched_2016_df['Population']
drug_sched_2016_df.head()


Schedule_Group,Zip,County,State,xYear,C2,C3,C4,C2-C4
0,90001,Los Angeles,CA,2016,843,201,1710,2754
1,90002,Los Angeles,CA,2016,357,767,1059,2183
2,90003,Los Angeles,CA,2016,1329,259,2440,4028
3,90004,Los Angeles,CA,2016,3582,1707,5002,10291
4,90005,Los Angeles,CA,2016,439,1403,1434,3276


In [219]:
drug_metrics_df = pd.read_csv("Resources/schedule_drugs_table2_metrics_by_year_and_zip.csv")

# create dataframe with only 2016 drug data
drug_metrics_2016_df = drug_metrics_df.loc[drug_metrics_df['xYear']==2016]

#drug_metrics_df.sort_values('Zip')
# what % of records are unique Zips
drug_metrics_2016_df['Zip'].nunique()/drug_metrics_2016_df['Zip'].count()

rx_count_2016_df = drug_metrics_2016_df[['Zip','Rx_count_Pat','Pat_count','RxCount_Total_Pat']]
rx_count_2016_df = rx_count_2016_df.rename(columns={'Rx_count_Pat':'Opioid Rx Count',
                                                    'Pat_count':'Pop w Opioid Rx',
                                                    'RxCount_Total_Pat':'Tot Opioid Rx Fills'})


# Merge controlled Rx with total Rx count
rx_data_df = rx_count_2016_df.merge(drug_sched_2016_df,on='Zip', how='outer')
rx_data_df = rx_data_df[['Zip','County','State','Opioid Rx Count','Pop w Opioid Rx','Tot Opioid Rx Fills','C2','C3','C4','C2-C4']]

rx_data_df.head()

,Zip,County,State,Opioid Rx Count,Pop w Opioid Rx,Tot Opioid Rx Fills,C2,C3,C4,C2-C4
0,90001,Los Angeles,CA,7879,4202,7717,843,201,1710,2754
1,90002,Los Angeles,CA,10927,5072,10701,357,767,1059,2183
2,90003,Los Angeles,CA,12814,6073,12512,1329,259,2440,4028
3,90004,Los Angeles,CA,7149,3987,6832,3582,1707,5002,10291
4,90005,Los Angeles,CA,4292,2342,4188,439,1403,1434,3276


In [226]:
ca_adi_df = pd.read_csv("Resources/CA_ADI_by_ZIP.csv")
ca_adi_df = ca_adi_df.rename(columns={'ADI_STATERNK':'ADI State Rank','ADI_NATRANK':'ADI Natl Rank'})
ca_adi_df.head()

,Zip,ADI State Rank,ADI Natl Rank
0,90001,8.007138,45.794018
1,90002,8.125497,50.517541
2,90003,7.972540,46.906751
3,90004,4.238019,28.200175
4,90005,7.225923,57.775878


In [227]:
# Read Tapestry segments by CA zip
tapestry_df = pd.read_csv("Resources/Tapestry_by_CA_Zip.csv")
tapestry_df = tapestry_df.rename(columns={'ZIP Code':'Zip','Dominant Tapestry Segment Number per Zip':'Tapestry Seg Nr',
                                         'Dominant Tapestry Segment Code per Zip':'Tapestry Seg Code',
                                         'Dominant Tapestry Segment Name per Zip':'Tapestry Seg Name',
                                         'Dominant Lifemode Code':'Lifemode Code',
                                         'Dominant Lifemode Group Name':'Lifemode Group'})
tapestry_df.head()

,Zip,NAME,Tapestry Seg Nr,Tapestry Seg Code,Tapestry Seg Name,Lifemode Code,Lifemode Group
0,90001.0,Los Angeles,61.0,13B,Las Casas,13.0,Next Wave
1,90002.0,Los Angeles,61.0,13B,Las Casas,13.0,Next Wave
2,90003.0,Los Angeles,61.0,13B,Las Casas,13.0,Next Wave
3,90004.0,Los Angeles,60.0,13A,International Marketplace,13.0,Next Wave
4,90005.0,Los Angeles,62.0,13C,NeWest Residents,13.0,Next Wave


In [228]:
ca_doi_df = pd.read_csv("Resources/CA_DOI_Underserved_Zips.csv")
ca_doi_df.head()

,Zip,CA DOI Underserved Flag
0,90001,Y
1,90002,Y
2,90003,Y
3,90004,Y
4,90005,Y


##  Merge all relevant data into one dataframe

In [229]:
# Merge all relevant data sets
# Census and hospital data
combo1_df = ca_census_demo_df.merge(ca_hosp_2016_df, left_on='Zip', right_on='pzip', how='inner')
# add Rx data
combo2_df = combo1_df.merge(rx_data_df, on='Zip', how='inner')
# add Area Deprivation Index
combo3_df = combo2_df.merge(ca_adi_df, on='Zip', how='inner')
# add Tapestry segmentation data
combo4_df = combo3_df.merge(tapestry_df, on='Zip', how='inner')
# add CA Dept of Insurance underserved zip code flag
combo5_df = combo4_df.merge(ca_doi_df, on='Zip', how='left')
combo5_df['CA DOI Underserved Flag'] = combo5_df['CA DOI Underserved Flag'].fillna('N')  # populates zip codes not in CA DOI list
combo5_df.count()

Zip                        1518
Total Pop                  1518
Male Pop                   1518
Pct Male                   1518
Female Pop                 1518
Pct Female                 1518
pzip                       1518
Year                       1518
AS Only                    1516
ED Only                    1518
Inpatient                  1515
Inpatient from ED          1517
County                     1518
State                      1518
Opioid Rx Count            1518
Pop w Opioid Rx            1518
Tot Opioid Rx Fills        1518
C2                         1518
C3                         1518
C4                         1518
C2-C4                      1518
ADI State Rank             1518
ADI Natl Rank              1518
NAME                       1518
Tapestry Seg Nr            1518
Tapestry Seg Code          1518
Tapestry Seg Name          1518
Lifemode Code              1517
Lifemode Group             1517
CA DOI Underserved Flag    1518
dtype: int64

In [237]:
# Add Ambulatory Surgery (AS), Emergency Dept (ED), Inpatient (discharges) and Inpatient from ED utilization rates per 1,000 pop
combo5_df['AS per 1,000'] = combo5_df['AS Only'] / combo5_df['Total Pop']*1000
combo5_df['ED per 1,000'] = combo5_df['ED Only'] / combo5_df['Total Pop']*1000
combo5_df['IP per 1,000'] = combo5_df['Inpatient'] / combo5_df['Total Pop']*1000
combo5_df['IP via ED per 1,000'] = combo5_df['Inpatient from ED'] / combo5_df['Total Pop']*1000

# Calculate drug utilization ratios per 1,000 population
combo5_df['C2 per 1,000'] = combo5_df['C2']/combo5_df['Total Pop']*1000
combo5_df['C3 per 1,000'] = combo5_df['C3']/combo5_df['Total Pop']*1000
combo5_df['C4 per 1,000'] = combo5_df['C4']/combo5_df['Total Pop']*1000
combo5_df['C2-C4 per 1,000'] = combo5_df['C2-C4']/combo5_df['Total Pop']*1000
combo5_df['Opioid Rx per 1,000'] = combo5_df['Opioid Rx Count']/combo5_df['Total Pop']*1000
combo5_df['Pop w Opioid Rx per 1,000'] = combo5_df['Pop w Opioid Rx']/combo5_df['Total Pop']*1000

## Clean final merged data and export to csv file

In [236]:
# Remove duplicate columns
combo6_df = combo5_df[['Zip','County','State','Year','Total Pop','Male Pop','Pct Male','Female Pop','Pct Female',
                       'ADI State Rank','ADI Natl Rank','CA DOI Underserved Flag','Tapestry Seg Nr','Tapestry Seg Code',
                       'Tapestry Seg Name','Lifemode Code','Lifemode Group','AS Only','ED Only','Inpatient',
                       'Inpatient from ED','Opioid Rx Count','Pop w Opioid Rx','Tot Opioid Rx Fills',
                       'C2','C3','C4','C2-C4','AS per 1,000','ED per 1,000','IP per 1,000','IP via ED per 1,000',
                       'C2 per 1,000','C3 per 1,000','C4 per 1,000','C2-C4 per 1,000','Opioid Rx per 1,000',
                       'Pop w Opioid Rx per 1,000']]

# Clean null and infinite values from data
combo7_df = combo6_df.replace([np.inf, -np.inf], np.nan)
combo8_df = combo7_df.dropna(axis=0)

# Save Clean Combined Data to csv file
combo8_df.to_csv(f"Combined_Hosp_Drug_Util.csv", encoding='utf-8')

combo8_df.count()

Zip                          1509
County                       1509
State                        1509
Year                         1509
Total Pop                    1509
Male Pop                     1509
Pct Male                     1509
Female Pop                   1509
Pct Female                   1509
ADI State Rank               1509
ADI Natl Rank                1509
CA DOI Underserved Flag      1509
Tapestry Seg Nr              1509
Tapestry Seg Code            1509
Tapestry Seg Name            1509
Lifemode Code                1509
Lifemode Group               1509
AS Only                      1509
ED Only                      1509
Inpatient                    1509
Inpatient from ED            1509
Opioid Rx Count              1509
Pop w Opioid Rx              1509
Tot Opioid Rx Fills          1509
C2                           1509
C3                           1509
C4                           1509
C2-C4                        1509
AS per 1,000                 1509
ED per 1,000  